# Notebook 1: Scraping Articles from FT.com  

**By:** El Mehdi Daoudi  
**For Deepsearch Labs task**   

---

This is the **first notebook** I created to show how I managed to **scrape articles from FT.com**.  

- I use **Selenium** because the site is **JavaScript-based**.  
- I showed how I **bypass the paywall** and also **handle Cloudflare protection**.  
- I save all the scraped data into **MongoDb**.  
- I also show how I schedule the scraper for **daily ingestion** so I always get fresh articles.  

---

This notebook is only focused on **data scraping**.  
The **second notebook** will handle **data analysis, visualization, and automated presentation**.


---

## 1- Import libraries

I started by importing the libraries I need , like:

- selenium : automating the browser..


- selenium_stealth : to hide the scraper so Cloudflare does not block me.


- pymongo : to save my data into MongoDB.


- time : to add waiting time between requests.

In [1]:
from selenium.webdriver.common.by import By
from selenium_stealth import stealth
from selenium import webdriver
from selenium.common.exceptions import NoSuchWindowException, WebDriverException, InvalidSessionIdException
from pymongo import MongoClient
import schedule
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

---
## 2- Connect to MongoDB

I connected to **MongoDB** and created a **database deepsearch_labs_db1** with a **collection articles** , so i can save th e scraped articles in this database.

In [14]:
client = MongoClient("mongodb://localhost:27017")
db = client["deepsearch_labs_db1"]
collection = db["articles"]

---
## 3- Initialize Driver

I created this function to initialize the driver. At first I tried to bypass the paywall by trying many thing like deleting the JavaScript part that show paywall, but it didn’t work. After searching, I found the extension Bypass Paywalls Chrome Clean Master and it worked fine. I used Brave as a browser because the Chrome version didn’t let me use the extension.

- In the options, I told Selenium to install the extension.

- I used stealth mode to bypass Cloudflare detection.


In [15]:
def init_driver():
    brave_binary = "C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe"
    extension_path = "C:/Users/Pro/Downloads/bypass-paywall/"

    options = webdriver.ChromeOptions()
    options.binary_location = brave_binary
    options.add_argument("--headless=new")  #running the browsers in background
    options.add_argument("--disable-gpu") 
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(f"--load-extension={extension_path}") #to install the extension "Bypass Paywalls Clean"

    driver = webdriver.Chrome(options=options)

    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True)
    return driver

---
## 4- Scrape Section

And this is the most important part, I created this function to scrape all articles from a given FT.com section. I can run it in daily mode (small scrape to avoid duplicates) or full mode (first-time scrape).

- I open the browser driver.

- I go through each page in the section (2 for daily, 20 for full).

- I collect all the article links.

- For each link, I open the page and extract details:

  - Title

  - Article body

  - Topic / Tag

  - Date & time

  - Authors

- I save the article in MongoDB only if it is not already in the database (to avoid duplicates).
- I handle errors and always close the browser in the end. 

In [16]:
def scrape_section(section, daily=False):

    driver = None
    inserted_count = 0  # count of inserted articles

    try:
        #browser driver 
        driver = init_driver()
        max_pages = 2 if daily else 21 #2 pages for daily scrape (avoid duplicates), 20 pages for first-time full scrape
        
        #loop in section pages
        for page in range(1, max_pages + 1):
            url = f"https://www.ft.com/{section}?page={page}"

            try:
                driver.get(url)
                time.sleep(3)  # time to load
            except (InvalidSessionIdException, NoSuchWindowException, WebDriverException):
                print(f"[{section}] Browser closed or invalid session")
                break

            # first i added this to find and collect article links on the page

            hrefs = []
            try:
                articles = driver.find_elements(By.CSS_SELECTOR, "div.o-teaser__content")
                for article in articles:
                    try:
                        header_a = article.find_element(By.CSS_SELECTOR, "div.o-teaser__heading a")
                        href = header_a.get_attribute("href")

                        if href and href.startswith("https://www.ft.com/content/"):
                            hrefs.append(href)
                    except:
                        continue
            except Exception as e:
                print(f"[{section}] Error collecting links on page {page}: {e}")
                continue
                

            # i added this loop to visit each link article and extract data

            for href in hrefs:
                try:
                    driver.get(href)
                    time.sleep(2)

                    # extract article details : title
                    try:
                        title = driver.find_element(By.CSS_SELECTOR, "span.headline__text").text.strip()
                    except:
                        title = None
                        
                        
                    # extract article details : article body
                    try:
                        article_body = driver.find_element(By.CSS_SELECTOR, "article#article-body").text.strip()
                    except:
                        article_body = None

                        
                    # extract article details : tag
                    try:
                        tag = driver.find_element(
                            By.CSS_SELECTOR,
                            "a.o-topper__topic.n-content-tag--with-follow.js-primary-theme"
                        ).text.strip()
                    except:
                        tag = None
                        
                        
                    # extract article details : datatime
                    try:
                        datetime_val = driver.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")
                    except:
                        datetime_val = None

                    # extract article details : authours
    
                    try:
                        author_elems = driver.find_elements(By.CSS_SELECTOR, "a.o3-editorial-typography-byline-author")
                        authors = ", ".join(
                            [a.text.strip() for a in author_elems if a.text.strip()]
                        ) if author_elems else None
                    except:
                        authors = None

                    #mongoDB document that i will send to DB
                    oc = {
                        "section": section,
                        "title": title,
                        "link": href,
                        "topic": tag,
                        "datetime": datetime_val,
                        "authors": authors,
                        "article": article_body
                    }

                    # here i added a conditional statement to insert only if not already in DB (to avoid duplicates articles , because we will scrape the pages daily , also some pages have the article link in most read and in search)
                    if not collection.find_one({"link": href}):
                        collection.insert_one(doc)
                        inserted_count += 1
                        print(f"[{section}] inserted: {title}")
                    else:
                        print(f"[{section}] skipped duplicate: {title}")
                        
                # I added this because I got exceptions when closing the browser
                except (InvalidSessionIdException, NoSuchWindowException, WebDriverException):
                    print(f"[{section}] browser closed")
                    break
                #I added this to catch any other unexpected scraping errors    
                except Exception as e:
                    print(f"[{section}] error scraping article {href}: {e}")

    finally:
        
        if driver:
            try:
                driver.quit()
            except:
                pass

        print(f"[{section}] browser closed. Scraped {inserted_count} articles")


# 5- Parallel scraping


I created this function to run a swarm of scrapers. It scrapes many FT.com sections at the same time to make it faster.

The sections are: world, companies, tech, markets, climate.

- I use ThreadPoolExecutor to run 5 scrapers in parallel.

- I can run it in daily mode (only the first pages after the full mode) or full mode (scraping 20 pages).

- It shows the start time and a message when it finishes

In [12]:
sections = ["world", "companies", "tech", "markets", "climate"]

def run_scraper(daily=False):
    print(f"Running scraper (daily={daily}) at {datetime.now()}")
    with ThreadPoolExecutor(max_workers=5) as executor:
        for section in sections:
            executor.submit(scrape_section, section, daily)
    print("finished")

## 6- Run Full Scrape & Daily ingestion

I used this part to run the scraper. First I run a full scrape (20 pages per section). After that, I run a daily scrape (only 2 pages) every morning at 10:00.

- run_scraper(daily=False) : full scrape.

- run_scraper(daily=True) : daily scrape (just the first two pages).

- I used schedule to run it every day at 10:00.

The loop keeps the scheduler running forever.

In [18]:
# full scrape (20 pages per section)
run_scraper(daily=False)

# Daily scraping ( after full scraping , daily scrape (only 2 pages) at 10:00 every morning + the scheduler running forever)

#schedule.every().day.at("10:00").do(run_scraper, daily=True)
#while True:
 #   schedule.run_pending()
  #  time.sleep(60)



Running scraper (daily=False) at 2025-08-25 03:07:31.596947
[world] inserted: France summons US ambassador over antisemitism claims
[tech] inserted: Sony is building a vast games empire: Can it keep control?
[markets] inserted: EU speeds up plans for digital euro after US stablecoin law
[climate] inserted: Green Climate Fund chief Mafalda Duarte: ‘We need a different scale of investments’
[tech] inserted: Spotify signals further price rises as it rolls out new services
[world] inserted: Iran’s supreme leader rules out direct talks with US
[markets] inserted: Hedge funds bet against European car firms as tariffs bite
[climate] inserted: Donald Trump’s attacks on renewables sector quash nearly $19bn worth of projects
[companies] inserted: HSBC Swiss unit culls wealthy Middle Eastern clients amid regulator scrutiny
[world] inserted: Mark Carney praises Donald Trump’s Ukraine peace push as he visits Kyiv
[tech] inserted: US to take 10% stake in troubled chipmaker Intel
[markets] inserted: 

[tech] inserted: Chinese semiconductor shares surge after DeepSeek gives boost to homegrown chips
[world] skipped duplicate: Tech stocks are sending a warning
[markets] inserted: Development finance: how to get more bang from fewer bucks
[companies] inserted: Keurig Dr Pepper nears $18bn deal to buy European coffee group JDE Peet’s
[climate] inserted: How we made it: southern Europe’s wild summer
[world] inserted: The myths that made Putin’s war
[tech] inserted: OnlyFans hands record $701mn dividend to owner ahead of sale
[markets] inserted: FirstFT: Powell’s moment at Jackson Hole
[companies] inserted: FirstFT: South Korea’s leftwing president set for talks with Trump on military alliance and trade
[climate] inserted: FT series: the Uninsurable World
[world] skipped duplicate: Keurig Dr Pepper nears $18bn deal to buy European coffee group JDE Peet’s
[tech] inserted: Elon Musk tried to enlist Mark Zuckerberg in $100bn bid for OpenAI
[markets] inserted: The return of ‘pump and dump’ sto

[companies] inserted: The lamentable decline of reading
[tech] inserted: Sympathy Tower Tokyo — a fantastical fable of identity, AI and ChatGPT
[climate] inserted: Ørsted shares sink on $9bn fundraising plan to shore up Trump-hit wind project
[world] skipped duplicate: Coca-Cola explores sale of Costa Coffee
[markets] skipped duplicate: BlackRock advises investors to put more money into hedge funds
[companies] inserted: Gold diggers follow the money
[climate] inserted: Big Oil heeds call to ‘drill, baby, drill’ in bet green transition will slow
[world] inserted: Notting Hill Carnival food sellers grapple with rising costs
[companies] skipped duplicate: TikTok to lay off hundreds of UK moderators as it shifts to AI
[markets] inserted: It’s not a default if you never ask for cash — right?
[tech] skipped duplicate: For stockpickers, AI is already both co-pilot and competitor
[world] skipped duplicate: Investors look to S&P’s ‘forgotten 493’ stocks as megacap tech wobbles
[climate] inserte

[climate] skipped duplicate: How one Greek island has a plan to help save the Mediterranean
[world] skipped duplicate: Is it time to sell your AI stocks?
[markets] inserted: Scott Bessent bets on stablecoins to bolster demand for Treasuries
[companies] inserted: Worldline: How to make a €1.6bn overdraft disappear
[tech] inserted: Opendoor’s next chief has a fixer-upper on their hands
[climate] skipped duplicate: Can the world’s beaches survive a sand shortage?
[tech] inserted: UK lags behind rest of world in tackling Big Tech, says Fortnite chief
[world] skipped duplicate: The myths that made Putin’s war
[companies] skipped duplicate: OnlyFans hands record $701mn dividend to owner ahead of sale
[markets] inserted: The Iranian connection: how China is importing oil from Russia
[climate] skipped duplicate: Good COP, worse COP
[tech] inserted: UK has ‘agreed to drop’ demand for access to Apple user data, says US
[world] skipped duplicate: UK lenders pull back from green mortgage market
[c

[tech] inserted: Investors grapple with Trump’s corporate America
[world] inserted: Canada to drop retaliatory tariffs on billions of dollars’ worth of US products
[companies] skipped duplicate: OpenAI and Anthropic clamp down on investment vehicles
[climate] skipped duplicate: Is there a case for climate optimism?
[markets] inserted: London’s IPO drought: why City bankers are pivoting
[tech] inserted: Smart glasses look better this time round
[world] inserted: The twilight of the central banking elite
[companies] skipped duplicate: Fortress JPMorgan: Jamie Dimon’s monument to American finance
[climate] skipped duplicate: For sun-and-sea holidays, October is the new August
[markets] inserted: Monzo plans UK mobile service as competition intensifies for big phone groups
[tech] inserted: Data of former UK ministers and Afghan refugees leaked after cyber attack
[companies] skipped duplicate: India dogged by Trump spotlight on Russian oil
[world] skipped duplicate: Government supporting fo

...

[world] skipped duplicate: The moneymen funding Big Tech’s AI dreams
[climate] inserted: EU cracks down on fast fashion and food waste
[world] inserted: Japan’s pacifist self-image is under attack from realpolitik
[climate] inserted: Most big polluters miss UN deadline for 2035 climate targets
[world] inserted: Tax data allays fears of non-dom exodus from UK
[climate] inserted: Senior Republicans seeking to tear up IRA enjoy $130bn investment spree
[world] inserted: Fake diploma scandal rattles faith in Turkish state
[climate] inserted: BP investors call for vote on any rowback on climate goals
[world] skipped duplicate: Four BCG staff quit Gaza aid project over early concerns
[world] skipped duplicate: ‘Showtime’s not here yet’: Cincinnati consumers shrug off Donald Trump’s tariffs
[climate] inserted: How each country’s emissions and climate goals compare
[world] skipped duplicate: What I witnessed at the battle of Orgreave
[climate] inserted: Japan’s $1.5bn bet on ultra-thin solar ce

[climate] inserted: Brussels to exempt most EU companies from carbon border tax
[world] skipped duplicate: FirstFT: US inflation data buoys global markets
[climate] inserted: Hottest January on record shocks scientists
[world] inserted: Where to eat the best tacos in New York
[climate] inserted: Starmer wants contentious North Sea oil and gas fields to go ahead
[world] inserted: Cafe Tropical: the bakery that Los Angeles brought back to life
[climate] inserted: Jeff Bezos’s $10bn Earth Fund cuts ties with climate group
[world] inserted: South Korea’s former first lady in solitary confinement after bribery arrest
[climate] inserted: European airlines push back hydrogen ambitions
[world] inserted: Benjamin Netanyahu’s allies turn on army chief as Gaza rift deepens
[world] inserted: Why governments still need ideology
[climate] inserted: Record temperatures highlight ‘Faustian bargain’, says leading scientist
[world] inserted: China targets 2 Lithuanian banks as EU relations sour over Ukr

[world] browser closed. Scraped 255 articles
[climate] inserted: Indonesia tests global carbon credits market linked to energy projects
[climate] inserted: Climate change increases threat of heat deaths in European cities
[climate] inserted: The coming great global land reshuffle
[climate] inserted: How the Davos crowd responded to Trump’s return
[climate] inserted: Spain warns far right is exploiting anger over climate disasters
[climate] inserted: China’s onetime ‘solar king’ struggles in US electric bus market
[climate] inserted: Climate catastrophes call for state intervention on insurance
[climate] inserted: Brussels under pressure to curb green agenda in response to Trump
[climate] inserted: Climate graphic of the week: Wild January weather assails the US
[climate] inserted: Donald Trump’s exit from Paris climate accord has ‘huge impact’, says Brazil’s COP30 chief
[climate] inserted: Why do people stay put when climate change jeopardises their homes?
[climate] inserted: Ed Miliba

[climate] inserted: Winds add new threat as deaths from LA fires reach 10
[climate] inserted: World breaches 1.5C global warming target for first time in 2024
[climate] inserted: BlackRock quits climate change group in latest green climbdown
[climate] inserted: Insurers brace for losses of up to $20bn from California wildfires
[climate] inserted: VW strains to hit EU emission targets as EV deliveries tumble
[climate] inserted: New year, new mandates: European aviation industry forced to buy greener fuels
[climate] inserted: Catastrophes cost world $320bn in 2024, reinsurer reports
[climate] inserted: Los Angeles wildfires rage out of control leaving at least 5 dead
[climate] inserted: How the small print of the Kyoto Protocol became a gripping piece of theatre
[climate] inserted: Biggest shipping nations back flat tax on emissions
[climate] inserted: Quarter of freshwater species face extinction risk, research finds
[climate] inserted: Green venture finance weathers the storm
[climate]

'schedule.every().day.at("10:00").do(run_scraper, daily=True)\nwhile True:\n    schedule.run_pending()\n    time.sleep(60)\n'